In [1]:
import tensorflow as tf
import numpy as np
from data import dataprep
from tensorflow.contrib import layers
from tensorflow.contrib.learn import *
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
from tensorflow.python.estimator.inputs import numpy_io

In [7]:
train_dict, test_dict = dataprep.mf_train_test()
rating_train = train_dict.pop("rating")
rating_test = test_dict.pop("rating")

In [3]:
model_params = dict(
    n_user=train_dict['user_id'].max() + 1,
    n_movie=train_dict['movie_id'].max() + 1,
    n_dim=20,
    reg_param=0.01,
    learning_rate=0.01,
)

In [4]:
def mf_function(features, targets, mode, params):
    users = features['user_id']
    movies = features['movie_id']
    ratings = targets

    with tf.variable_scope("embedding"):
        user_weight = tf.get_variable("user_w"
                                      , shape=[params['n_user'], params['n_dim']]
                                      , dtype=tf.float32
                                      , initializer=layers.xavier_initializer())
        user_bias = tf.get_variable("user_b"
                                    , shape=[params['n_user']]
                                    , dtype=tf.float32
                                    , initializer=tf.zeros_initializer)
        movie_weight = tf.get_variable("movie_w"
                                       , shape=[params['n_movie'], params['n_dim']]
                                       , dtype=tf.float32
                                       , initializer=layers.xavier_initializer())
        movie_bias = tf.get_variable("movie_b"
                                     , shape=[params['n_movie']]
                                     , dtype=tf.float32
                                     , initializer=tf.zeros_initializer)

    with tf.name_scope("inference"):
        user_emb = tf.nn.embedding_lookup(user_weight, users)
        u_b = tf.nn.embedding_lookup(user_bias, users)
        movie_emb = tf.nn.embedding_lookup(movie_weight, movies)
        m_b = tf.nn.embedding_lookup(movie_bias, movies)
        pred = tf.reduce_sum(tf.multiply(user_emb, movie_emb), 1) + u_b + m_b

    with tf.name_scope("loss"):
        reg_loss = layers.apply_regularization(layers.l2_regularizer(scale=params['reg_param']),
                                               weights_list=[user_weight, movie_weight])
        loss = tf.nn.l2_loss(pred - ratings) + reg_loss
        rmse = tf.sqrt(tf.reduce_mean(tf.pow(pred - ratings, 2)))
        

    eval_metric_ops = {'rmse': rmse}
    train_op = layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=params["learning_rate"],
        optimizer=tf.train.AdamOptimizer,
        summaries=[
            "learning_rate",
            "loss",
            "gradients",
            "gradient_norm",
        ])

    return ModelFnOps(mode, pred, loss, train_op, eval_metric_ops)

In [11]:
# input queue for training
train_input_fn = numpy_io.numpy_input_fn(
    x=train_dict, y=rating_train, batch_size=10000, shuffle=True, num_epochs=None)
# input queue for evaluation on test data
test_input_fn = numpy_io.numpy_input_fn(
    x=test_dict, y=rating_test, batch_size=rating_test.shape[0], shuffle=False, num_epochs=None)

validation_monitor = monitors.ValidationMonitor(input_fn=test_input_fn, eval_steps=1, every_n_steps=10,
                                                name='test')

mf_estimator = Estimator(
    model_fn=mf_function,
    params=model_params,
    model_dir='model/_summary/mf_test_1',
    config=RunConfig(save_checkpoints_secs=5))

mf_estimator.fit(input_fn=train_input_fn, steps=1000, monitors=[validation_monitor])

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Using config: {'_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11cea4908>, '_is_chief': True, '_save_summary_steps': 100, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_task_type': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 5, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_environment': 'local', '_tf_random_seed': None, '_evaluation_master': '', '_model_dir': None, '_save_checkpoints_steps': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into model/_summary/mf_test_1/model.ckpt.
INFO:tensorflow:loss = 68286.1, step = 1
INFO:tensorflow:Starting evaluation at 2017-06-03-14:05:52
INFO:tensorflow:Restoring parameters from model/_summary/mf_test_1/model.ckpt-1
INFO:tensorflow:Evaluation [1/1]
INFO:te

INFO:tensorflow:Validation (step 310): global_step = 309, loss = 9135.28, rmse = 0.954212
INFO:tensorflow:Saving checkpoints for 337 into model/_summary/mf_test_1/model.ckpt.
INFO:tensorflow:Starting evaluation at 2017-06-03-14:06:52
INFO:tensorflow:Restoring parameters from model/_summary/mf_test_1/model.ckpt-337
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-06-03-14:06:54
INFO:tensorflow:Saving dict for global step 337: global_step = 337, loss = 9283.89, rmse = 0.961874
INFO:tensorflow:Validation (step 340): global_step = 337, loss = 9283.89, rmse = 0.961874
INFO:tensorflow:Saving checkpoints for 365 into model/_summary/mf_test_1/model.ckpt.
INFO:tensorflow:Starting evaluation at 2017-06-03-14:06:58
INFO:tensorflow:Restoring parameters from model/_summary/mf_test_1/model.ckpt-365
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-06-03-14:07:00
INFO:tensorflow:Saving dict for global step 365: global_step = 365, loss = 9461.98, 

KeyboardInterrupt: 